<a href="https://colab.research.google.com/github/dhan16/colabs/blob/master/covid19opendata/WikiData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## WikiData

In [19]:
# Sparql functions
import requests
import pandas as pd

ENDPOINT = "https://query.wikidata.org/sparql"


def wiki_data(sparql):
  res = requests.get(ENDPOINT, params = {'format': 'json', 'query': sparql})
  return res.json()


def wikidata_to_dataframe(json):
  results = json["results"]["bindings"]
  # column names we draw from the first result
  cols = [ val for val in results[0] ]
  rows = []
  for result in results:
      values = [ result[val]["value"] for val in result ]
      rows.append(values)
  return pd.DataFrame(rows, columns=cols)


In [20]:
sparql = """
SELECT ?place ?placeLabel ?class ?classLabel
WHERE
{
  ?place wdt:P31/wdt:P279* wd:Q12479774.
  ?place wdt:P31 ?class.
  ?class wdt:P279 wd:Q12479774.
  SERVICE wikibase:label {
    bd:serviceParam wikibase:language "en" .
  }
}
"""
res = wiki_data(sparql)
df = wikidata_to_dataframe(res)
df
# res

,class,place,placeLabel,classLabel
0,http://www.wikidata.org/entity/Q3191695,http://www.wikidata.org/entity/Q12488339,Banggai Laut,regency of Indonesia
1,http://www.wikidata.org/entity/Q3191695,http://www.wikidata.org/entity/Q11214749,Sidenreng Rappang,regency of Indonesia
2,http://www.wikidata.org/entity/Q3191695,http://www.wikidata.org/entity/Q4201892,Katingan,regency of Indonesia
3,http://www.wikidata.org/entity/Q3191695,http://www.wikidata.org/entity/Q4201768,Maluku Tengah,regency of Indonesia
4,http://www.wikidata.org/entity/Q3191695,http://www.wikidata.org/entity/Q11491,Bangli,regency of Indonesia
...,...,...,...,...
512,http://www.wikidata.org/entity/Q3199141,http://www.wikidata.org/entity/Q7253,Padang,city of Indonesia
513,http://www.wikidata.org/entity/Q3199141,http://www.wikidata.org/entity/Q7248,Bukittinggi,city of Indonesia
514,http://www.wikidata.org/entity/Q3199141,http://www.wikidata.org/entity/Q5989,Tebing Tinggi,city of Indonesia
515,http://www.wikidata.org/entity/Q3199141,http://www.wikidata.org/entity/Q5987,Tanjungbalai,city of Indonesia


In [21]:
df.classLabel.unique()

kabkota_to_regiontype = {
    'regency of Indonesia' : 'Regency',
    'administrative regency of Indonesia' : 'Regency',
    'city of Indonesia': 'City',
    'administrative city of Indonesia': 'City',
}

df['regiontype'] = df.apply(lambda r: kabkota_to_regiontype[r.classLabel], axis=1)
wiki_df = df
wiki_df

,class,place,placeLabel,classLabel,regiontype
0,http://www.wikidata.org/entity/Q3191695,http://www.wikidata.org/entity/Q12488339,Banggai Laut,regency of Indonesia,Regency
1,http://www.wikidata.org/entity/Q3191695,http://www.wikidata.org/entity/Q11214749,Sidenreng Rappang,regency of Indonesia,Regency
2,http://www.wikidata.org/entity/Q3191695,http://www.wikidata.org/entity/Q4201892,Katingan,regency of Indonesia,Regency
3,http://www.wikidata.org/entity/Q3191695,http://www.wikidata.org/entity/Q4201768,Maluku Tengah,regency of Indonesia,Regency
4,http://www.wikidata.org/entity/Q3191695,http://www.wikidata.org/entity/Q11491,Bangli,regency of Indonesia,Regency
...,...,...,...,...,...
512,http://www.wikidata.org/entity/Q3199141,http://www.wikidata.org/entity/Q7253,Padang,city of Indonesia,City
513,http://www.wikidata.org/entity/Q3199141,http://www.wikidata.org/entity/Q7248,Bukittinggi,city of Indonesia,City
514,http://www.wikidata.org/entity/Q3199141,http://www.wikidata.org/entity/Q5989,Tebing Tinggi,city of Indonesia,City
515,http://www.wikidata.org/entity/Q3199141,http://www.wikidata.org/entity/Q5987,Tanjungbalai,city of Indonesia,City


## Sheet

In [ ]:
!pip install --upgrade gspread

In [22]:
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

sheet_url = 'https://docs.google.com/spreadsheets/d/1FJJXiGuOb5nXrjJeV3QcHNhTo38YdcsTIFl29mWDIqI/edit#gid=2006070746'
worksheet = gc.open_by_url(sheet_url).worksheet('Kode Kota')
rows = worksheet.get_all_values()

# Convert to a DataFrame and render.
import pandas as pd
df = pd.DataFrame.from_records(rows[2:], columns=rows[1])


In [23]:
kabkota_to_regiontype = {
    'Kab.' : 'Regency', 
    'Kota': 'City', 
    'zTam' : 'zTam'
}

df['regiontype'] = df.apply(lambda r: kabkota_to_regiontype[r.KabKota], axis=1)

sheet_df = df
sheet_df

,ID Provinsi,Provinsi,ID Kota,KabKota,Kota,regiontype
0,1,Aceh (NAD),258,Kab.,Aceh Barat,Regency
1,1,Aceh (NAD),259,Kab.,Aceh Barat Daya,Regency
2,1,Aceh (NAD),260,Kab.,Aceh Besar,Regency
3,1,Aceh (NAD),261,Kab.,Aceh Jaya,Regency
4,1,Aceh (NAD),262,Kab.,Aceh Selatan,Regency
...,...,...,...,...,...,...
557,34,Sumatera Utara,492,Kab.,Tapanuli Selatan,Regency
558,34,Sumatera Utara,493,Kab.,Tapanuli Tengah,Regency
559,34,Sumatera Utara,494,Kab.,Tapanuli Utara,Regency
560,34,Sumatera Utara,495,Kota,Tebing Tinggi,City


## Sheet vs WikiData

In [25]:
spellings = {
    'Labuhanbatu': 'Labuhan Batu'
}

# Find rows in sheet_df with no matches in wiki_df
df = sheet_df.merge(wiki_df, how='left', left_on=['Kota', 'regiontype'], right_on=['placeLabel', 'regiontype']) 
# missing = df[df['placeLabel'].isnull() & ~df['KabKota'].isin(['zTam'])]
missing = df[df['placeLabel'].isnull()]
missing

,ID Provinsi,Provinsi,ID Kota,KabKota,Kota,regiontype,class,place,placeLabel,classLabel
23,1,Aceh (NAD),1011,zTam,Luar Provinsi Aceh,zTam,NaN,NaN,NaN,NaN
24,1,Aceh (NAD),1013,zTam,Luar Negeri (Aceh),zTam,NaN,NaN,NaN,NaN
25,1,Aceh (NAD),1012,zTam,Belum Diverifikasi (Aceh),zTam,NaN,NaN,NaN,NaN
26,2,Bali,1021,zTam,WNA (Bali),zTam,NaN,NaN,NaN,NaN
27,2,Bali,1022,zTam,Luar Provinsi Bali,zTam,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
537,34,Sumatera Utara,472,Kab.,Labuhanbatu,Regency,NaN,NaN,NaN,NaN
551,34,Sumatera Utara,486,Kota,Pematang Siantar,City,NaN,NaN,NaN,NaN
556,34,Sumatera Utara,491,Kota,Tanjung Balai,City,NaN,NaN,NaN,NaN
558,34,Sumatera Utara,493,Kab.,Tapanuli Tengah,Regency,NaN,NaN,NaN,NaN
